<a href="https://colab.research.google.com/github/olcaykursun/ML/blob/main/neuralnets/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical

# Load the dataset
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Normalize the pixel values
x_train, x_test = x_train / 255.0, x_test / 255.0

# Expand dimensions to include channel information
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

# let's say we remove class 8 and 9 for this example
classes_to_use = range(8)
idx_train = np.isin(y_train, classes_to_use)
idx_test = np.isin(y_test, classes_to_use)

x_train, y_train = x_train[idx_train], y_train[idx_train]
x_test, y_test = x_test[idx_test], y_test[idx_test]

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, num_classes=8)
y_test = to_categorical(y_test, num_classes=8)


In [ ]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(8, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=5, validation_split=0.2)


2024-04-02 12:13:57.305668: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-04-02 12:13:57.305703: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-04-02 12:13:57.305713: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-04-02 12:13:57.305990: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-04-02 12:13:57.306241: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/5


2024-04-02 12:13:58.498144: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1200/1200 [==============================] - 16s 10ms/step - loss: 0.5200 - accuracy: 0.8020 - val_loss: 0.3856 - val_accuracy: 0.8615
Epoch 2/5
1200/1200 [==============================] - 12s 10ms/step - loss: 0.3704 - accuracy: 0.8626 - val_loss: 0.3602 - val_accuracy: 0.8672
Epoch 3/5
1200/1200 [==============================] - 12s 10ms/step - loss: 0.3343 - accuracy: 0.8759 - val_loss: 0.4326 - val_accuracy: 0.8454
Epoch 4/5
1200/1200 [==============================] - 12s 10ms/step - loss: 0.3178 - accuracy: 0.8860 - val_loss: 0.3548 - val_accuracy: 0.8724
Epoch 5/5
1200/1200 [==============================] - 12s 10ms/step - loss: 0.3125 - accuracy: 0.8899 - val_loss: 0.3456 - val_accuracy: 0.8811


In [ ]:
feature_extractor = models.Model(inputs=model.input, outputs=model.layers[-3].output)


In [ ]:
# Extract features
features_train = feature_extractor.predict(x_train)
features_test = feature_extractor.predict(x_test)

# New model
new_model = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(features_train.shape[1],)),
    layers.Dense(8, activation='softmax')  # Assuming you still work with 8 classes
])

new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the new model
new_model.fit(features_train, y_train, epochs=5, validation_split=0.2)



250/250 [==============================] - 0s 981us/step
Epoch 1/5
1200/1200 [==============================] - 10s 9ms/step - loss: 0.3806 - accuracy: 0.8634 - val_loss: 0.3279 - val_accuracy: 0.8810
Epoch 2/5
1200/1200 [==============================] - 11s 9ms/step - loss: 0.2957 - accuracy: 0.8932 - val_loss: 0.3190 - val_accuracy: 0.8854
Epoch 3/5
1200/1200 [==============================] - 10s 9ms/step - loss: 0.2782 - accuracy: 0.8997 - val_loss: 0.3099 - val_accuracy: 0.8932
Epoch 4/5
1200/1200 [==============================] - 11s 9ms/step - loss: 0.2667 - accuracy: 0.9041 - val_loss: 0.3313 - val_accuracy: 0.8855
Epoch 5/5
1200/1200 [==============================] - 10s 9ms/step - loss: 0.2592 - accuracy: 0.9065 - val_loss: 0.3113 - val_accuracy: 0.8910


In [ ]:
# Evaluate the new model on the test set
test_loss, test_acc = new_model.evaluate(features_test, y_test)

print(f"Test loss: {test_loss}")
print(f"Test accuracy: {test_acc}")


250/250 [==============================] - 2s 7ms/step - loss: 0.3316 - accuracy: 0.8869
Test loss: 0.3315610885620117
Test accuracy: 0.8868749737739563


In [ ]:
from tensorflow.keras.metrics import Accuracy

# Use the model to predict the classes of the test set
predictions = new_model.predict(features_test)

# The predictions are in the form of probabilities for each class.
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(y_test, axis=1)

# Calculate accuracy by comparing predicted and true classes
accuracy = np.mean(predicted_classes == true_classes)

print(f"Test accuracy: {accuracy}")

250/250 [==============================] - 0s 1ms/step
Test accuracy: 0.886875


In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Decision tree
tree = DecisionTreeClassifier()
tree.fit(features_train, np.argmax(y_train, axis=1))  # Fit to the max class index or use the original labels

# Evaluate the decision tree
accuracy = tree.score(features_test, np.argmax(y_test, axis=1))
print(f"Decision Tree Accuracy: {accuracy}")


250/250 [==============================] - 0s 970us/step
Decision Tree Accuracy: 0.797125
